In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

In [3]:
from google.colab import files
myfile = files.upload()

Saving train.csv to train.csv


In [33]:
train = pd.read_csv("train.csv")
test = pd.read_csv('test.csv')
train.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [34]:
nCar = train.shape[0]
nVar = train.shape[1]
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar: 2000 nVar: 21


In [35]:
train_num = train[['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 
                 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time']]
train_bool = train[['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi']]

In [36]:
train_num.head()

,battery_power,clock_speed,fc,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time
0,842,2.2,1,7,0.6,188,2,2,20,756,2549,9,7,19
1,1021,0.5,0,53,0.7,136,3,6,905,1988,2631,17,3,7
2,563,0.5,2,41,0.9,145,5,6,1263,1716,2603,11,2,9
3,615,2.5,0,10,0.8,131,6,9,1216,1786,2769,16,8,11
4,1821,1.2,13,44,0.6,141,2,14,1208,1212,1411,8,2,15


In [37]:
train_bool.head()

,blue,dual_sim,four_g,three_g,touch_screen,wifi
0,0,0,0,0,0,1
1,1,1,1,1,1,0
2,1,1,1,1,1,0
3,1,0,0,1,0,0
4,1,0,1,1,1,0


In [38]:
origin_feature = train.drop('price_range', 1)
target = train['price_range']

x_train, x_test, y_train, y_test = train_test_split(origin_feature, target, test_size = 0.3)

<ipython-input-38-d72343a7e5ea>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  origin_feature = train.drop('price_range', 1)


In [41]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1400, 20), (600, 20), (1400,), (600,))

In [43]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
dt_clf.fit(x_train , y_train)
pred = dt_clf.predict(x_test)

accuracy_score(y_test , pred)

0.775

In [44]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth' : [ 6, 8 ,10, 12, 16 ,20, 24]
}

grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', 
                       cv=5, return_train_score=True)

grid_cv.fit(x_train , y_train)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=4, random_state=13),
             param_grid={'max_depth': [6, 8, 10, 12, 16, 20, 24]},
             return_train_score=True, scoring='accuracy')

In [45]:
cv_results_df = pd.DataFrame(grid_cv.cv_results_)
cv_results_df.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_max_depth', 'params', 'split0_test_score', 'split1_test_score',
       'split2_test_score', 'split3_test_score', 'split4_test_score',
       'mean_test_score', 'std_test_score', 'rank_test_score',
       'split0_train_score', 'split1_train_score', 'split2_train_score',
       'split3_train_score', 'split4_train_score', 'mean_train_score',
       'std_train_score'],
      dtype='object')

In [46]:
grid_cv.best_score_

0.8392857142857142

In [47]:
grid_cv.best_params_

{'max_depth': 6}

In [48]:
cv_results_df[['param_max_depth', 'mean_test_score', 'mean_train_score']]

,param_max_depth,mean_test_score,mean_train_score
0,6,0.839286,0.931250
1,8,0.833571,0.977857
2,10,0.834286,0.996071
3,12,0.829286,1.000000
4,16,0.829286,1.000000
5,20,0.829286,1.000000
6,24,0.829286,1.000000


In [49]:
max_depths = [ 6, 8 ,10, 12, 16 ,20, 24]

for depth in max_depths:
    dt_clf = DecisionTreeClassifier(max_depth=depth, random_state=156)
    dt_clf.fit(x_train , y_train)
    pred = dt_clf.predict(x_test)
    accuracy = accuracy_score(y_test , pred)
    print('Max_Depth =', depth, ', Accuracy =', accuracy)

Max_Depth = 6 , Accuracy = 0.845
Max_Depth = 8 , Accuracy = 0.8483333333333334
Max_Depth = 10 , Accuracy = 0.8433333333333334
Max_Depth = 12 , Accuracy = 0.8316666666666667
Max_Depth = 16 , Accuracy = 0.8316666666666667
Max_Depth = 20 , Accuracy = 0.8316666666666667
Max_Depth = 24 , Accuracy = 0.8316666666666667


In [50]:
best_df_clf = grid_cv.best_estimator_
pred1 = best_df_clf.predict(x_test)

accuracy_score(y_test , pred1)

0.845